### NLBSE'24 RoBERTa Baseline

In [1]:
BASE_MODEL = "roberta-base"
RANDOM_SEED = 42
OUTPUT_PATH = 'output/roberta'

!mkdir -p $OUTPUT_PATH

In [2]:
from datasets import Dataset

ds = Dataset.from_csv({ "train": "data/issues_train.csv", "test": "data/issues_test.csv" })
ds

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['repo', 'created_at', 'label', 'title', 'body'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['repo', 'created_at', 'label', 'title', 'body'],
        num_rows: 1500
    })
})

In [3]:
repos = ds["train"].unique("repo")
print(repos)

['facebook/react', 'tensorflow/tensorflow', 'microsoft/vscode', 'bitcoin/bitcoin', 'opencv/opencv']


In [4]:
ds["train"].to_pandas().groupby(["repo", "label"]).size().unstack(fill_value=0)

label,bug,feature,question
repo,,,
bitcoin/bitcoin,100,100,100
facebook/react,100,100,100
microsoft/vscode,100,100,100
opencv/opencv,100,100,100
tensorflow/tensorflow,100,100,100


In [5]:
import re

label2id = { "bug": 0, "feature": 1, "question": 2 }
id2label = { 0: "bug", 1: "feature", 2: "question" }

def process_dataset(example):

    example['label'] = label2id[example['label']]

    # concatenate title and body
    text = (example['title'] or "") + " " + (example['body'] or "")

    # Remove strings between triple quotes
    text = re.sub(r'```.*?```', ' ', text, flags=re.DOTALL)

    # Remove new lines
    text = re.sub(r'\n', ' ', text)

    # Remove links
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)

    # Remove digits
    text = re.sub(r'\d+', ' ', text)

    # Remove special characters except the question marks
    text = re.sub(r'[^a-zA-Z0-9?\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    example['text'] = text

    return example

ds = ds.shuffle(seed=RANDOM_SEED)
ds = ds.map(process_dataset)
ds = ds.select_columns(['repo', 'label', 'text'])


In [6]:
import wandb
from datetime import datetime
import evaluate
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import RobertaTokenizerFast, RobertaConfig, RobertaForSequenceClassification, TextClassificationPipeline
import numpy as np

group = datetime.utcnow().replace(microsecond=0).isoformat()

metric = evaluate.load("f1")

def repo_eq(repo: str):
    return lambda example: example['repo'] == repo

references = {}
predictions = {}

for repo in repos:
    wandb.init(
        project="NLBSE'24 Issue Report Classification - RoBERTa", 
        group=group,
        name=repo, 
    )

    device="cuda"
    truncation=True
    padding="max_length"
    max_length=512

    tokenizer = RobertaTokenizerFast.from_pretrained(BASE_MODEL)

    config = RobertaConfig.from_pretrained(BASE_MODEL, num_labels=3)

    model = RobertaForSequenceClassification.from_pretrained(BASE_MODEL, config=config)
  
    classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=device, truncation=truncation, padding=padding, max_length=max_length)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    def preprocess_function(examples):
      return tokenizer(examples['text'], truncation=truncation, padding=padding, max_length=max_length)

    train_set = ds.filter(repo_eq(repo))["train"]
    train_set = train_set.map(preprocess_function, batched=True)
    train_set.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    test_set = ds.filter(repo_eq(repo))["test"]
    references[repo] = [id2label[id] for id in test_set['label']]
    test_set = test_set.map(preprocess_function, batched=True)
    test_set.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels, average="weighted")

    training_args = TrainingArguments(
        output_dir=f"{OUTPUT_PATH}/{repo.replace('/', '-')}",
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="no",
        report_to="wandb",
        run_name=repo,
        seed=RANDOM_SEED,
        num_train_epochs=10,
        per_device_train_batch_size=16,
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
    )

    trainer.train()

    predictions[repo] = classifier(test_set['text'])
    predictions[repo] = [pred['label'] for pred in predictions[repo]]
    predictions[repo] = [model.config.label2id[label] for label in predictions[repo]]
    predictions[repo] = [id2label[id] for id in predictions[repo]]
    
    wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rafaelkallis. Use `wandb login --relogin` to force relogin


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.889000,0.683371,0.505019
2,0.682300,0.613906,0.639636
3,0.546800,0.547347,0.789345
4,0.432200,0.551864,0.760293
5,0.311100,0.650390,0.750040
6,0.113300,0.607732,0.839547
7,0.076900,0.873719,0.827400
8,0.021200,0.854718,0.841026
9,0.003900,0.868349,0.843438
10,0.003000,0.881902,0.844094


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/f1,▁▄▇▆▆█████
eval/loss,▄▂▁▁▃▂█▇██
eval/runtime,▁▂▄▅▆▆▇▇▇█
eval/samples_per_second,█▆▅▄▃▃▂▂▂▁
eval/steps_per_second,█▇▅▄▃▃▂▂▂▁
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▆▅▄▃▂▂▁▁▁
train/total_flos,▁
train/train_loss,▁


...

eval/f1,▁▆▇█▇█▇▇██
eval/loss,▄▂▁▂▂▅▇██▇
eval/runtime,▁▃▅▆▇▇██▇▇
eval/samples_per_second,█▆▄▃▂▂▁▁▂▂
eval/steps_per_second,█▆▄▃▂▂▁▁▂▂
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▆▄▃▃▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 300/300 [00:00<00:00, 2020.36 examples/s]
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,1.075100,0.968205,0.593196
2,0.939800,0.817039,0.561382
3,0.727100,0.832950,0.642537
4,0.632600,0.727519,0.628608
5,0.564300,0.866814,0.556447
6,0.472500,0.845133,0.640861
7,0.358900,0.818358,0.677320
8,0.322300,0.881727,0.670452
9,0.249400,0.882026,0.722118
10,0.222000,0.845077,0.742175


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/f1,▂▁▄▄▁▄▆▅▇█
eval/loss,█▄▄▁▅▄▄▅▅▄
eval/runtime,▁▃▄▅▆▇▇█▇█
eval/samples_per_second,█▆▅▄▃▂▂▁▂▁
eval/steps_per_second,█▆▅▄▃▂▂▁▂▁
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▇▅▄▄▃▂▂▁▁
train/total_flos,▁
train/train_loss,▁


In [7]:
from sklearn.metrics import classification_report
from numpy import mean

results = {}
metrics = ['precision', 'recall', 'f1-score']
labels = ['bug', 'feature', 'question']

for repo in repos:
  results[repo] = classification_report(references[repo], predictions[repo], digits=4, output_dict=True)
  results[repo]['average'] = results[repo]['weighted avg']
  results[repo] = {label: {metric: results[repo][label][metric] for metric in metrics} for label in labels + ['average']}

results['overall'] = {label: {metric: mean([results[repo][label][metric] for repo in repos]) for metric in metrics} for label in labels + ['average']}


In [8]:
import json
import os

output_file_name = 'results.json'

with open(os.path.join(OUTPUT_PATH, output_file_name), 'w') as fp:
    json.dump(results, fp, indent=2)

print(f"Repository{' '*15}Label     Precision  Recall     F1")
for repo in repos + ['overall']:
  print("-"*63)
  for label in labels + ['average']:
    out = f"{repo:<25}{label:<10}"
    for metric in metrics:
      out += f"{results[repo][label][metric]:<10.4f} "
    print(out)

Repository               Label     Precision  Recall     F1
---------------------------------------------------------------
facebook/react           bug       0.8909     0.9800     0.9333     
facebook/react           feature   0.8200     0.8200     0.8200     
facebook/react           question  0.8222     0.7400     0.7789     
facebook/react           average   0.8444     0.8467     0.8441     
---------------------------------------------------------------
tensorflow/tensorflow    bug       0.9659     0.8500     0.9043     
tensorflow/tensorflow    feature   0.8641     0.8900     0.8768     
tensorflow/tensorflow    question  0.7615     0.8300     0.7943     
tensorflow/tensorflow    average   0.8638     0.8567     0.8585     
---------------------------------------------------------------
microsoft/vscode         bug       0.7895     0.7500     0.7692     
microsoft/vscode         feature   0.6803     0.8300     0.7477     
microsoft/vscode         question  0.8554     0.7100     0